In [ ]:
# This is R code in juptyer lab, to install R kernel in jupter, open R session inside your oen anaconda environment
# type in the following:
# install.packages("devtools")
# devtools::install_github("IRkernel/IRkernel")
# IRkernel::installspec()
# then restart jupterlab
# ref: https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/
library(rxnorm)
library(dplyr)
library(data.table)
library(reshape2)
library(stringi)
home_directory = "/home/hchan2/AKI/AKI_Python/"

In [ ]:
RIGHT = function(x,n){
  substring(x,nchar(x)-n+1)
}

In [ ]:
# read data
rxnorm_data_ext <- readRDS(paste0(home_directory,"rxnorm2atcdict.rda"))
rxnorm_data_new <- read.delim(paste0(home_directory,"rxnormtmp.csv"), sep=",", stringsAsFactors=F, header = FALSE)

In [ ]:
# find new query needed
rxnorm_data_q <- rxnorm_data_new %>% filter(!V1 %in% rxnorm_data_ext$V1)

In [ ]:
#query new data
rxnorm_data_new_q <- rxnorm_data_q %>% mutate(V2=sapply(V1, rxnorm::get_atc))

In [ ]:
#add new query to database
rxnorm_data_ext <- rbind(rxnorm_data_ext, rxnorm_data_new_q)
saveRDS(rxnorm_data_ext, paste0(home_directory,"rxnorm2atcdict.rda"))

In [ ]:
# process output for python
rxnorm_data_out <- rxnorm_data_ext %>% filter(V1 %in% rxnorm_data_new$V1)
rxnorm_data_out["id"] <- rownames(rxnorm_data_out)
flatten_list <- melt(rxnorm_data_out$V2) %>% mutate(id=as.character(L1)) %>% 
                left_join(rxnorm_data_out, by="id") %>%
                select(V1, value) %>% mutate(V2=as.character(value)) %>% 
                mutate(V3=V2) %>%
#                mutate(V3=RIGHT(V2,1)) %>%
                select(V1, V3) %>% rename(c("Rxcui"="V1", "ATC4th"="V3")) %>% unique
write.csv(x=flatten_list, file='/home/hchan2/AKI/AKI_Python/rxnorm_out.csv', row.names = FALSE)